# Installing of necessary packages

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

# Loading of Documents

In [3]:
loader = TextLoader('Refund Policy Document.txt', encoding = 'UTF-8')
data = loader.load()
data

[Document(metadata={'source': 'Refund Policy Document.txt'}, page_content='1. Application for Returns/Refunds\n Subject to the terms and conditions in this Refunds and Return Policy and the Terms of Service, Buyer may apply for return of the purchased items (“Item”) and/or refund.\n For the avoidance of doubt, an application for a refund and/or return may be made whether prior to or after the expiry of the Shopee Guarantee Period as stated in the Terms of Service. Users may communicate with each other privately to resolve their differences prior, during or after using Shopee Guarantee.\n 2. Application for the Return of an Item\n Buyer may only apply for\n(i) the refund of the Item where such Item has not been received by Buyer (“Unreceived Items”), or\n(ii) the refund and/or return of an Item received by Buyer (“Received Items”) in the following circumstances:\na) Buyer received an empty and/or suspicious parcel;\nb) The Item received was incomplete (missing quantity or accessories);\

In [4]:
# data[0].page_content

In [5]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 1000,
#     chunk_overlap = 50,
#     length_function = len
#     # Existing args
# )

# documents = text_splitter.split_documents(data)

# documents

In [6]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(data)

C:\Users\Owent\AppData\Local\Temp\ipykernel_49068\3924431901.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\Owent\AppData\Local\Temp\ipykernel_49068\3924431901.py:1: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())


In [7]:
print("Number of chunks created: ", len(documents))

for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

Number of chunks created:  5

CHUNK : 1
1. Application for Returns/Refunds
 Subject to the terms and conditions in this Refunds and Return Policy and the Terms of Service, Buyer may apply for return of the purchased items (“Item”) and/or refund. For the avoidance of doubt, an application for a refund and/or return may be made whether prior to or after the expiry of the Shopee Guarantee Period as stated in the Terms of Service. Users may communicate with each other privately to resolve their differences prior, during or after using Shopee Guarantee. 2. Application for the Return of an Item
 Buyer may only apply for
(i) the refund of the Item where such Item has not been received by Buyer (“Unreceived Items”), or
(ii) the refund and/or return of an Item received by Buyer (“Received Items”) in the following circumstances:
a) Buyer received an empty and/or suspicious parcel;
b) The Item received was incomplete (missing quantity or accessories);
c) Seller has delivered an Item that does not

# Creating of Chat model

In [8]:
# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)

C:\Users\Owent\AppData\Local\Temp\ipykernel_49068\1995767516.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [9]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# retrieved_docs = retriever.invoke("What is the refund policy?")

# print(retrieved_docs[0])

In [10]:
# Define the Ollama LLM
llm = Ollama(model="mistral", temperature=0)

C:\Users\Owent\AppData\Local\Temp\ipykernel_49068\1894498698.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral", temperature=0)


In [18]:
prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you do not know the answer, just say that "I am not too sure. Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details" but do not make up an answer on your own.
3. For the conditions below, you should reply with "The question is not related to Shopee's Refund Policy":
    a. The question is completely irrelevant to Shopee's Refund Policy
    b. The question does not mention anything about a Refund policy
    c. If you are unsure if the question is relevant or not
    d. If the context provided does not give any good information related to the question

Context: {context}

Question: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )

In [20]:
print(qa("How are you?")["result"])
print(qa("What is Shopee's Refund Policy?")['result'])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you do not know the answer, just say that "I am not too sure. Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details" but do not make up an answer on your own.
3. For the conditions below, you should reply with "The question is not related to Shopee's Refund Policy":
    a. The question is completely irrelevant to Shopee's Refund Policy
    b. The question does not mention anything about a Refund policy
    c. If you are unsure if the question is relevant or not
    d. If the context provided does not give any good information related to the question

Context: Context:
content:Communication Between Buyer and Seller
 Shopee encourages Users to communicate with each other in the event where a problem arises in a transaction. As Shopee is a platform for Users to c